# ETL script to read data from an excel sheet with multiple sheets to a cosmos db with each sheet in their own container

#### Author: @Shaun De Ponte
#### 16 November 2021

In [1]:
# import all the various libraries
import pandas as pd
import json
from zipfile import ZipFile
from bs4 import BeautifulSoup
from azure.cosmos import CosmosClient
import azure.cosmos.exceptions as exceptions
from azure.cosmos.partition_key import PartitionKey
import os

In [2]:
os.environ['COSMOS_URI'] = 'https://localhost:8081'
os.environ['COSMOS_KEY'] = 'C2y6yDjf5/R+ob0N8A7Cgv30VRDJIWEHLM+4QDU5DE2nQ9nDuVTqobD4b8mGGyPMbIZnqyMsEcaGQy67XIw/Jw=='

In [3]:
# instantiate the cosmos db client
url = os.environ['COSMOS_URI']
key = os.environ['COSMOS_KEY']

client = CosmosClient(url, credential=key)

# name file variable
file = 'ianna.xlsx'
xls = pd.ExcelFile(file)

# scrape the excel sheet for sheet names
with ZipFile(file) as zipped_file:
    summary = zipped_file.open(r'xl/workbook.xml').read()
soup = BeautifulSoup(summary, "xml")
# store the sheet names in a list
sheets = [sheet.get("name") for sheet in soup.find_all("sheet")]

# declare some variables
data_map = []
dict = {}
inc = 0

# create a data map that would be used elsewhere
for i in range(len(sheets)):
    inc += 1
    dict['id'] = inc
    dict['filter'] = sheets[i]
    data_map.append(dict.copy())
    
# create a json reponse that would be used to store the data in cosmos
data = {}
for i in range(len(sheets)):
    df = pd.read_excel(xls, sheets[i])
    result = json.loads(json.dumps(list(df.T.to_dict().values())))
    data[sheets[i]] = result
    
# create id's for all the objects
final_data = {}
for x in data:
    for i in range(len(data_map)):
        if data_map[i]['filter'] == x:
            inc = 0
            temp_lst = data[x]
            for j in range(len(temp_lst)):
                inc += 1
                temp_lst[j]['id'] = str(inc)
            final_data[data_map[i]['filter']] = temp_lst

C:\Users\Shaun\AppData\Local\Programs\Orange\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [4]:
# check the data is good
print(final_data)

{'IA_Filter_ER_PM': [{'Entity_Ruler': 'Law enforcement officers',
   'Label': 'PERSON',
   'PM': 'warrant',
   'Impacted_Area': 'Legal',
   'id': '1'},
  {'Entity_Ruler': 'Local Municipality',
   'Label': 'ORG',
   'PM': 'property rates',
   'Impacted_Area': 'Informational',
   'id': '2'},
  {'Entity_Ruler': 'Local Municipality',
   'Label': 'ORG',
   'PM': 'rates',
   'Impacted_Area': 'Informational',
   'id': '3'},
  {'Entity_Ruler': 'Municipality',
   'Label': 'ORG',
   'PM': 'rates',
   'Impacted_Area': 'Informational',
   'id': '4'},
  {'Entity_Ruler': 'Local Municipality',
   'Label': 'ORG',
   'PM': 'valuation rolls',
   'Impacted_Area': 'Informational',
   'id': '5'},
  {'Entity_Ruler': 'Local Municipality',
   'Label': 'ORG',
   'PM': 'property tax rates',
   'Impacted_Area': 'Informational',
   'id': '6'},
  {'Entity_Ruler': 'Banks Act Directive',
   'Label': 'LAW',
   'PM': 'capital buffer',
   'Impacted_Area': 'Treasury and Capital Management',
   'id': '7'},
  {'Entity_Rul

In [5]:
# create a partition key
partition_key = PartitionKey(path='/id', kind='Hash')
# initialize the database
db = client.get_database_client('ImpactAreaDB')

#store the excel sheets in the database and create a new container for each sheet
for key, val in final_data.items():
    for i in range(len(sheets)):
        if key == sheets[i]:
            db.create_container(id=key, partition_key=partition_key)
            container = db.get_container_client(key)
            print(sheets[i])
            for j in range(len(val)):
                print(val[j])
                container.create_item(body=val[j])


C:\Users\Shaun\AppData\Local\Programs\Orange\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


IA_Filter_ER_PM
{'Entity_Ruler': 'Law enforcement officers', 'Label': 'PERSON', 'PM': 'warrant', 'Impacted_Area': 'Legal', 'id': '1'}
{'Entity_Ruler': 'Local Municipality', 'Label': 'ORG', 'PM': 'property rates', 'Impacted_Area': 'Informational', 'id': '2'}
{'Entity_Ruler': 'Local Municipality', 'Label': 'ORG', 'PM': 'rates', 'Impacted_Area': 'Informational', 'id': '3'}
{'Entity_Ruler': 'Municipality', 'Label': 'ORG', 'PM': 'rates', 'Impacted_Area': 'Informational', 'id': '4'}
{'Entity_Ruler': 'Local Municipality', 'Label': 'ORG', 'PM': 'valuation rolls', 'Impacted_Area': 'Informational', 'id': '5'}
{'Entity_Ruler': 'Local Municipality', 'Label': 'ORG', 'PM': 'property tax rates', 'Impacted_Area': 'Informational', 'id': '6'}
{'Entity_Ruler': 'Banks Act Directive', 'Label': 'LAW', 'PM': 'capital buffer', 'Impacted_Area': 'Treasury and Capital Management', 'id': '7'}
{'Entity_Ruler': 'Local Municipality', 'Label': 'ORG', 'PM': 'Land Use', 'Impacted_Area': 'Informational', 'id': '8'}
{'En

C:\Users\Shaun\AppData\Local\Programs\Orange\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Shaun\AppData\Local\Programs\Orange\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Shaun\AppData\Local\Programs\Orange\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Shaun\A

{'Entity_Ruler': 'City of Cape Town', 'Label': 'ORG', 'PM': 'valuation rolls', 'Impacted_Area': 'Informational', 'id': '12'}
{'Entity_Ruler': 'City of Mbombela', 'Label': 'ORG', 'PM': 'rates', 'Impacted_Area': 'Informational', 'id': '13'}
{'Entity_Ruler': 'City of Tshwane', 'Label': 'ORG', 'PM': 'rates', 'Impacted_Area': 'Informational', 'id': '14'}
{'Entity_Ruler': 'High Court', 'Label': 'ORG', 'PM': 'Divorce orders', 'Impacted_Area': 'Human Capital', 'id': '15'}
{'Entity_Ruler': 'Endumeni Municipality', 'Label': 'ORG', 'PM': 'rates', 'Impacted_Area': 'Informational', 'id': '16'}
{'Entity_Ruler': 'Ngqushwa Municipality', 'Label': 'GPE', 'PM': 'rates', 'Impacted_Area': 'Informational', 'id': '17'}
{'Entity_Ruler': 'Local Municipality', 'Label': 'ORG', 'PM': 'by-law', 'Impacted_Area': 'Informational', 'id': '18'}
{'Entity_Ruler': 'Local Municipality', 'Label': 'ORG', 'PM': 'by-laws', 'Impacted_Area': 'Informational', 'id': '19'}
{'Entity_Ruler': 'Local Municipality', 'Label': 'ORG', 'PM

C:\Users\Shaun\AppData\Local\Programs\Orange\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Shaun\AppData\Local\Programs\Orange\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Shaun\AppData\Local\Programs\Orange\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Shaun\A

...

C:\Users\Shaun\AppData\Local\Programs\Orange\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Shaun\AppData\Local\Programs\Orange\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Shaun\AppData\Local\Programs\Orange\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Shaun\A


{'Impacted Area': 'Human Capital', 'Representative ID': -1, 'Representative Name': -1, 'id': '12'}
{'Impacted Area': 'Informational', 'Representative ID': -1, 'Representative Name': -1, 'id': '13'}
{'Impacted Area': 'Insurance Distribution and Offshore Services', 'Representative ID': -1, 'Representative Name': -1, 'id': '14'}
{'Impacted Area': 'IT', 'Representative ID': -1, 'Representative Name': -1, 'id': '15'}
{'Impacted Area': 'Legal', 'Representative ID': -1, 'Representative Name': -1, 'id': '16'}
{'Impacted Area': 'Lending', 'Representative ID': -1, 'Representative Name': -1, 'id': '17'}
{'Impacted Area': 'Money Laundering Surveillance', 'Representative ID': -1, 'Representative Name': -1, 'id': '18'}

C:\Users\Shaun\AppData\Local\Programs\Orange\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Shaun\AppData\Local\Programs\Orange\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Shaun\AppData\Local\Programs\Orange\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Shaun\A


{'Impacted Area': 'PBB', 'Representative ID': -1, 'Representative Name': -1, 'id': '19'}
{'Impacted Area': 'Securities and Market Abuse', 'Representative ID': -1, 'Representative Name': -1, 'id': '20'}
{'Impacted Area': 'Tax', 'Representative ID': -1, 'Representative Name': -1, 'id': '21'}
{'Impacted Area': 'Treasury and Capital Management', 'Representative ID': -1, 'Representative Name': -1, 'id': '22'}


C:\Users\Shaun\AppData\Local\Programs\Orange\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Shaun\AppData\Local\Programs\Orange\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Shaun\AppData\Local\Programs\Orange\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\Shaun\A